# Import library
those are the requirement for the application

In [3]:
!pip install youtube-transcript-api google-generativeai

In [13]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
import google.generativeai as genai
import time
from youtube_transcript_api import YouTubeTranscriptApi
import re
from IPython.display import Markdown, display

# load the google API key 

In [5]:
# Load environment variables in a file called .env
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [6]:
# Start Google API

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
MODEL = "gemini-1.5-flash"
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
patterns = [
        r"(?:v=|/)([a-zA-Z0-9_-]{11})",  # Matches v=... or /...
        r"(?:embed/|v/|watch\?v=|youtu.be/|/v/)([a-zA-Z0-9_-]{11})", # Matches embed, v/, watch?v=, youtu.be, /v/
        r"youtu\.be/([a-zA-Z0-9_-]{11})", # Matches youtu.be short links
        r"(?:/shorts/|/live/)([a-zA-Z0-9_-]{11})", # Matches youtube shorts or live
    ]

In [10]:
# Define the youtube transcript class 
class YTVideoTranscripter:
    def __init__(self, url, languages=["en"]):
        self.url = url
        self.language = languages
        self.video_id = self.extract_video_id()
        self.transcription = self.get_transcript()
        self.systemPrompt = """
        You are a helpful assistant who provides concise and accurate summaries of text. Your task is to:
        
        - Capture the key points of the content.
        - Keep the summary brief and easy to understand.
        - Avoid summarizing overly lengthy texts or breaking them into excessively short summaries.
        - Use bullet points where appropriate to enhance clarity and structure.
        """
        
    def extract_video_id(self):
        """
        Extracts the YouTube video ID from a given URL.
        Supports both regular and shortened URLs.
        """
        # Regular expression to match YouTube video URL and extract the video ID
        regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|\S*\?v=)|(?:youtu\.be\/))([a-zA-Z0-9_-]{11})"
        match = re.match(regex, self.url)
        
        if match:
            self.match = match
            return match.group(1)
        else:
            raise ValueError("Invalid YouTube URL")

    def get_transcript(self):
        try:
            # Try to get the transcript in the desired language (Indonesian by default)
            transcript = YouTubeTranscriptApi.get_transcript(self.video_id, languages=[self.language])
            # Join all the 'text' fields into a single string
            return " ".join([item['text'] for item in transcript])
        except Exception as e:
            print(f"Error fetching transcript: {e}")
            return None
            
    def transcriptSummarizer(self):
        model = genai.GenerativeModel(model_name=MODEL,system_instruction=self.systemPrompt)
        response = model.generate_content(self.transcription)
        return response.text
        
        
    def __str__(self):
        return f"Video ID: {self.video_id}"

In [11]:
# Example usage
video_url = "https://www.youtube.com/watch?v=LUttsadgSEY"

yt_video = YTVideoTranscripter(video_url, languages="en")
print(len(yt_video.transcription))

9243


In [15]:
yt_video.transcriptSummarizer()

"This video presents five AI frameworks useful for building AI applications:\n\n*   **LangChain:** A Python-based framework for working with LLMs, simplifying tasks like prompt templates, output parsing, and managing vector databases.  It facilitates creating chains of operations.\n\n*   **LangFlow:** A visual, low-code editor similar to LangChain, allowing users to build LLM-based apps via drag-and-drop functionality.\n\n*   **Ollama:** A free, open-source tool for running LLMs locally on your computer, eliminating reliance on third-party services. Requires high-end hardware for optimal performance.\n\n*   **LlamaIndex:** A Python framework focused on handling large datasets for AI applications, offering features like document extraction and text chunking.  It emphasizes working with enterprise-grade data.\n\n*   **Hugging Face Transformers:** An open-source Python module simplifying the use of transformer-based models for various tasks (NLP, audio, video processing), offering pre-tra